<a href="https://colab.research.google.com/github/2003vivek/2003vivek/blob/main/beer_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/train.csv')
df.sample(3)

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct,review/timeUnix,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
26949,3743,6.8,33624,14879,Hoppin' To Heaven IPA,American IPA,2.5,3.0,2.0,2.0,2.0,Pours an orange beer with a massive amount of ...,"{'min': 50, 'hour': 16, 'mday': 9, 'sec': 31, ...",1304959831,NaN,NaN,NaN,NaN,Beerandraiderfan
20257,43673,3.5,41294,1,Pilsner Urquell 3.5%,Czech Pilsener,2.5,2.5,3.5,2.5,3.0,"Light gold beer, totally clear, light white he...","{'min': 18, 'hour': 12, 'mday': 11, 'sec': 34,...",1278850714,1.080573e+09,"Sep 14, 1980",337762800.0,Male,jrallen34
29097,14877,10.6,47658,1199,Founders CBS Imperial Stout,American Double / Imperial Stout,4.0,4.0,4.0,3.5,3.0,Finally....I got my hands on one of these bad ...,"{'min': 55, 'hour': 3, 'mday': 12, 'sec': 12, ...",1321070112,NaN,NaN,NaN,NaN,byrdman16


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37500 entries, 0 to 37499
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              37500 non-null  int64  
 1   beer/ABV           37500 non-null  float64
 2   beer/beerId        37500 non-null  int64  
 3   beer/brewerId      37500 non-null  int64  
 4   beer/name          37500 non-null  object 
 5   beer/style         37500 non-null  object 
 6   review/appearance  37500 non-null  float64
 7   review/aroma       37500 non-null  float64
 8   review/overall     37500 non-null  float64
 9   review/palate      37500 non-null  float64
 10  review/taste       37500 non-null  float64
 11  review/text        37490 non-null  object 
 12  review/timeStruct  37500 non-null  object 
 13  review/timeUnix    37500 non-null  int64  
 14  user/ageInSeconds  7856 non-null   float64
 15  user/birthdayRaw   7856 non-null   object 
 16  user/birthdayUnix  785

In [9]:
df.corr()['review/overall']

<ipython-input-9-92de16046b77>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()['review/overall']


index                0.054493
beer/ABV             0.197005
beer/beerId         -0.021981
beer/brewerId       -0.084932
review/appearance    0.498733
review/aroma         0.616117
review/overall       1.000000
review/palate        0.697220
review/taste         0.785220
review/timeUnix      0.065383
user/ageInSeconds   -0.045642
user/birthdayUnix    0.045642
Name: review/overall, dtype: float64

In [40]:
new_df=df[['beer/ABV','review/appearance','review/aroma','review/palate','review/taste','review/text','review/overall']]

In [41]:
new_df.sample(5)

,beer/ABV,review/appearance,review/aroma,review/palate,review/taste,review/text,review/overall
4532,10.2,4.0,4.5,4.0,4.0,A- A very very dark amber liquid. Ever so slig...,3.5
6754,11.2,5.0,4.5,5.0,5.0,Bottle dated 2/25/11. Got a 4pk at Best Yet in...,5.0
13454,6.5,3.0,4.0,3.0,3.5,12oz. bottle. Thanks to Alexgash for the oppor...,3.5
37330,3.5,2.5,2.0,2.0,2.0,12 oz bottle\t\tA pale macro yellow with a fiz...,3.0
11056,6.5,4.0,4.5,4.0,4.5,Are you kidding me? I've recently discovered F...,4.0


In [42]:
new_df.isnull().sum()

beer/ABV              0
review/appearance     0
review/aroma          0
review/palate         0
review/taste          0
review/text          10
review/overall        0
dtype: int64

In [43]:
new_df=new_df.dropna()

In [44]:
new_df.isnull().sum()

beer/ABV             0
review/appearance    0
review/aroma         0
review/palate        0
review/taste         0
review/text          0
review/overall       0
dtype: int64

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,accuracy_score

In [50]:
text=new_df['review/text']
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(text)

In [52]:
x_numericals=new_df.drop(['review/text','review/overall'],axis=1)

In [56]:
from scipy.sparse import hstack
data=hstack((x_numericals,X_text))
y=new_df['review/overall']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

In [58]:
model=RandomForestRegressor(random_state=42)

In [59]:
model.fit(X_train,y_train)

RandomForestRegressor(random_state=42)

In [60]:
y_pred=model.predict(X_test)

In [68]:
mse=mean_squared_error(y_test,y_pred)
print(f'mean squared error is : {mse}')

mean squared error is : 0.15478542611363033


In [67]:
import pickle
with open('model_pickle','wb') as f:
  pickle.dump(model,f)

In [73]:
text = 'its really amazing to have it'
text_vectorized = vectorizer.transform([text]).toarray().flatten()
unseen_data = [6.0, 3.0, 4.0, 3.0, 3.5] + text_vectorized.tolist()
model.predict([unseen_data])


array([3.32])